In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
dataset = pd.read_csv('twitter_racism_parsed_dataset.csv')
dataset

,index,id,Text,Annotation,oh_label
0,5.770000e+17,5.770000e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1
1,5.410000e+17,5.410000e+17,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,none,0
2,5.680000e+17,5.680000e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1
3,5.770000e+17,5.770000e+17,"Finally I'm all caught up, and that sudden dea...",none,0
4,5.710000e+17,5.710000e+17,@carolinesinders @herecomesfran *hugs*,none,0
...,...,...,...,...,...
13464,4.510000e+17,4.510000e+17,@RICANROLL You think your reply is clever? Wom...,none,0
13465,5.750000e+17,5.750000e+17,@asem_1994 ISIS beheads people. Saudi beheads ...,racism,1
13466,5.760000e+17,5.760000e+17,#mkr NOOOOO!!! I wanted Kat and Andre to lose!!!,none,0
13467,5.580000e+17,5.580000e+17,RT @MumtazCeltik: @WhiteHouse @VP \n\n#Kobane ...,none,0


In [13]:
dt_transformed = dataset[['oh_label', 'Text']]
y = dt_transformed.iloc[:, :-1].values
y

array([[1],
       [0],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [14]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
y = np.array(ct.fit_transform(y))

In [15]:
print(y)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [16]:
y_df = pd.DataFrame(y)
y_hate = np.array(y_df[0])
y_offensive = np.array(y_df[1])

In [17]:
print(y_hate)
print(y_offensive)

[0. 1. 0. ... 1. 1. 1.]
[1. 0. 1. ... 0. 0. 0.]


In [19]:
corpus = []
for i in range (0, 13469):
    review = re.sub("[^a-zA-Z]", ' ', dt_transformed['Text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

In [20]:
cv = CountVectorizer(max_features = 2000)
x = cv.fit_transform(corpus).toarray()

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y_hate, test_size = 0.30, random_state = 0)

#ERRORRRRRRRR CAH

In [23]:
classifier_np = GaussianNB()
classifier_np.fit(x_train, y_train)

GaussianNB()

In [24]:
classifier_lr = LogisticRegression(random_state = 0)
classifier_lr.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [25]:
classifier_svm = svm.SVC()
classifier_svm.fit(x_train, y_train)

SVC()

In [28]:
from sklearn.metrics import confusion_matrix
y_pred_np = classifier_np.predict(x_test)
cm = confusion_matrix(y_test, y_pred_np)
print(cm)

[[ 491  107]
 [1126 2317]]


In [29]:
y_pred_svm = classifier_svm.predict(x_test)
cm = confusion_matrix(y_test, y_pred_svm)
print(cm)

[[ 414  184]
 [  90 3353]]


In [30]:
y_pred_lr = classifier_lr.predict(x_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(cm)

[[ 408  190]
 [ 103 3340]]


In [33]:
from sklearn.metrics import accuracy_score
svm_score = accuracy_score(y_test, y_pred_svm)
lr_score = accuracy_score(y_test, y_pred_lr)
np_score = accuracy_score(y_test, y_pred_np)

print('Support Vector Machine Accuracy:', str(svm_score))
print('Logistic Regression Accuracy:', str(lr_score))
print('Naive Bayes Accuracy:', str(np_score))

Support Vector Machine Accuracy: 0.9321950012373175
Logistic Regression Accuracy: 0.9274931947537738
Naive Bayes Accuracy: 0.6948775055679287


In [38]:
import pickle
filename = 'model_SVM'
pickle.dump(classifier_svm, open(filename, 'wb'))

In [39]:
load_model = pickle.load(open(filename, 'rb'))
result = load_model.score(x_test, y_test)
print(result)

0.9321950012373175
